## Importamos librerias

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Cargamos los datos

    En este caso vamos a realizar un análisis de la jornada 37 para poder detectar que fichajes serían los más optimos de cara a la jornada del campeonato para tratar de obtener la mejor puntuación posible.

In [2]:
journey = pd.read_csv('data/pruebas/comunio_J37.csv')
clas = pd.read_excel('data/classification_J_37.xlsx')

In [3]:
journey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584 entries, 0 to 583
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Team_id              584 non-null    int64  
 1   Team                 584 non-null    object 
 2   Player               584 non-null    object 
 3   Position             584 non-null    object 
 4   Matchs               584 non-null    int64  
 5   On_start_%           584 non-null    int64  
 6   Goals                584 non-null    int64  
 7   Assists              584 non-null    int64  
 8   Total_Points         584 non-null    int64  
 9   Points_Average       584 non-null    float64
 10  Last_5_games_points  584 non-null    object 
 11  J_33                 584 non-null    int64  
 12  J_34                 584 non-null    int64  
 13  J_35                 584 non-null    int64  
 14  J_36                 584 non-null    int64  
 15  J_37                 584 non-null    int

In [4]:
clas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20 non-null     int64 
 1   Position    20 non-null     int64 
 2   Team        20 non-null     object
 3   Games       20 non-null     int64 
 4   Won         20 non-null     int64 
 5   Draw        20 non-null     int64 
 6   Lost        20 non-null     int64 
 7   Goals S.    20 non-null     int64 
 8   Goals R.    20 non-null     int64 
 9   GD          20 non-null     int64 
 10  PTS         20 non-null     int64 
dtypes: int64(10), object(1)
memory usage: 1.8+ KB


In [5]:
clas = clas.drop('Unnamed: 0', axis=1)

*Con los datos cargados vamos a realizar las transformaciones y filtros necesarios para obtener la tabla que necesitamos para el dashboard*

In [6]:
journey = journey.drop(['Last_5_games_points'], axis=1)

In [7]:
gk = journey.loc[journey['Position']=='PT'].sort_values('Total_Points', ascending=False)[:5]
df = journey.loc[journey['Position']=='DF'].sort_values('Total_Points', ascending=False)[:10]
md = journey.loc[journey['Position']=='MD'].sort_values('Total_Points', ascending=False)[:10]
dl = journey.loc[journey['Position']=='DL'].sort_values('Total_Points', ascending=False)[:10]

In [8]:
pos = [gk, df, md, dl]

In [9]:
top_players = pd.DataFrame()

for p in pos:
    top_players = top_players.append(pd.DataFrame(p))
    
top_players.head()

C:\Users\botic\AppData\Local\Temp\ipykernel_16948\3117522028.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_players = top_players.append(pd.DataFrame(p))
C:\Users\botic\AppData\Local\Temp\ipykernel_16948\3117522028.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_players = top_players.append(pd.DataFrame(p))
C:\Users\botic\AppData\Local\Temp\ipykernel_16948\3117522028.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_players = top_players.append(pd.DataFrame(p))
C:\Users\botic\AppData\Local\Temp\ipykernel_16948\3117522028.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_players = top_players.append(pd.DataFr

,Team_id,Team,Player,Position,Matchs,On_start_%,Goals,Assists,Total_Points,Points_Average,J_33,J_34,J_35,J_36,J_37,Avg_last_5_games,Value
471,17,Cádiz CF,Jeremías Ledesma,PT,37,100,0,0,184,5.0,7,6,8,2,9,6.4,3030000
200,8,Real Madrid CF,Courtois,PT,35,95,0,0,182,5.2,2,12,0,6,0,4.0,5760000
349,13,Real Sociedad de Fútbol,Alex Remiro,PT,36,92,1,1,181,5.3,7,8,4,6,3,5.6,4230000
526,19,RC Celta de Vigo,Matías Dituro,PT,37,100,0,0,178,4.8,5,7,6,-1,6,4.6,3940000
49,3,CA Osasuna,Sergio Herrera,PT,35,95,0,0,173,4.9,1,3,2,0,6,2.4,3760000


## Ahora vamos ha realizar un poco ingeniería de datos para obtener algunas columnas que nos puedas ayudar al estudio

*En primer lugar vamos a añadir una columna para obtener el precio por puntos totales de cada jugador*

In [10]:
top_players['Price_per_Total_Points'] = (top_players['Value']/top_players.Total_Points).apply(lambda x: '{0:.2f}'.format(x))

*Ahora hacemos lo mismo pero únicamente calculando el valor por punto obtenido en las últimas cinco jornadas*

In [11]:
top_players['Price_per_Points_Last_5_J'] = (top_players['Value']/ (top_players['J_33']
                                                    +top_players['J_34']
                                                    + top_players['J_35']
                                                    +top_players['J_36']
                                                    +top_players['J_37'])).apply(lambda x: '{0:.2f}'.format(x))
    

In [12]:
top_players.head()

,Team_id,Team,Player,Position,Matchs,On_start_%,Goals,Assists,Total_Points,Points_Average,J_33,J_34,J_35,J_36,J_37,Avg_last_5_games,Value,Price_per_Total_Points,Price_per_Points_Last_5_J
471,17,Cádiz CF,Jeremías Ledesma,PT,37,100,0,0,184,5.0,7,6,8,2,9,6.4,3030000,16467.39,94687.50
200,8,Real Madrid CF,Courtois,PT,35,95,0,0,182,5.2,2,12,0,6,0,4.0,5760000,31648.35,288000.00
349,13,Real Sociedad de Fútbol,Alex Remiro,PT,36,92,1,1,181,5.3,7,8,4,6,3,5.6,4230000,23370.17,151071.43
526,19,RC Celta de Vigo,Matías Dituro,PT,37,100,0,0,178,4.8,5,7,6,-1,6,4.6,3940000,22134.83,171304.35
49,3,CA Osasuna,Sergio Herrera,PT,35,95,0,0,173,4.9,1,3,2,0,6,2.4,3760000,21734.10,313333.33


### Una vez que tenemos la tabla con los datos la exportamos a excel para poder cargarla en Tableau 

In [13]:
with pd.ExcelWriter('data/comunio_tableau_J37.xlsx') as writer:  
    top_players.to_excel(writer, sheet_name='top_players')
    clas.to_excel(writer, sheet_name='classification')